## hedging mode does not perform as well as non-hedging 

> If hedging is True, allow trades in both directions simultaneously. If False, the opposite-facing orders first close existing trades in a FIFO manner.

In [ ]:
import pandas as pd
import humanize

In [ ]:
input_path = '../../tmp-data/ss_range_bars.no_zero_close'
known_false_signals = False


In [ ]:
df = pd.read_csv(f'{input_path}.csv', parse_dates=['timestamp'], index_col='timestamp')
df

In [ ]:
def get_start_end_idx_for_period(df: pd.DataFrame, resample_arg: str = 'M') -> list[str]:
        # resample the dataframe into monthly periods
        groups = df.resample(resample_arg)
        # create a list of tuples containing the start and end indices for each period
        start_end_indices = []
        # iterate over each group (period) in the resampled DataFrame
        for period_start, group_df in groups:
            # get the start and end indices of the rows within the current period
            period_start = group_df.index[0]
            period_end = group_df.index[-1]
            # add the start and end indices to the list
            start_end_indices.append((period_start, period_end))
        return start_end_indices

In [ ]:
start_end_indices = get_start_end_idx_for_period(df)
start_end_indices

In [ ]:
import random
from backtesting import Strategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarStrategyPreCalc(Strategy):

    per_trade_risk = 0.02
    trades_buy_open_at_time_of_buy = []
    trades_sell_open_at_time_of_sell = []
    trades_open_per_tick = []
    
    
    def init(self):
       self.false_buys = 0
       self.false_sells = 0
       self.trades_buy_open_at_time_of_buy = []
       self.trades_sell_open_at_time_of_sell = []
       self.trades_open_per_tick = []

    # Stop Loss Price = Liquidation Price - (Liquidation Price - Entry Price) / Leverage * Stop Loss Ratio
    def next(self):
        ts = self.data.index[-1]
        current_close = self.data.Close[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
     
        coin_toss = random.choice([True, False])    

        order_placed = False
        if pre_calc_signal == 1:
            order_placed = True        
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal == -1:
            order_placed = True    
            self.sell(size=self.per_trade_risk, sl=current_close + range_size , tp=current_close - potential_profit)

        # order counts    
        if order_placed:    
          is_short_count = len(list(filter(lambda t: t.is_short, self.trades)))
          is_long_count = len(list(filter(lambda t: t.is_long, self.trades)))
          self.trades_buy_open_at_time_of_buy.append(is_long_count)
          self.trades_sell_open_at_time_of_sell.append(is_short_count)
        is_short_count = len(list(filter(lambda t: t.is_short, self.trades)))
        is_long_count = len(list(filter(lambda t: t.is_long, self.trades)))
        self.trades_open_per_tick.append({'timestamp': ts, 'buy': is_long_count, 'sell': is_short_count})

In [ ]:


def backtest_period(strat, df, all_stats, start, end, per_trade_risk=0.02, leverage=20, hedging=False, crypto_cash_adjustment=100000):
    from backtesting import Backtest
    strat.per_trade_risk = per_trade_risk
    bt = Backtest(df, strat, hedging=hedging, cash=crypto_cash_adjustment, margin=1/leverage, commission=(0.04 / 100), exclusive_orders=False)
    stats = bt.run()

    # collect stats
    stats_cp = stats.copy()
    stats_cp['start'] = start
    stats_cp['end'] = end
    all_stats.append(stats_cp)

    # argument stats
    false_buys = stats._strategy.false_buys
    false_sells = stats._strategy.false_sells
    trades = stats['# Trades']
    trade_return_percs = stats._trades['ReturnPct']
    return_percentage = stats['Return [%]']
    win_rate_percentage = stats['Win Rate [%]']
    profit_factor = stats['Profit Factor']
    net_profit = (stats['Equity Final [$]'] - crypto_cash_adjustment)
    equity = stats['Equity Final [$]']
    max_drawdown_percentage = stats['Max. Drawdown [%]']
    average_drawdown_percentage = stats['Avg. Drawdown [%]']
    percentage_false_trades = round(((false_buys + false_sells) / int(trades)) * 100, 2)
    return {
        "trades_buy_open_at_time_of_buy": [i for i in stats._strategy.trades_buy_open_at_time_of_buy if i != 0],
        "trades_sell_open_at_time_of_sell": [i for i in stats._strategy.trades_sell_open_at_time_of_sell if i != 0],
        "trades_open_per_tick": stats._strategy.trades_open_per_tick,
        "trade_return_percentages": trade_return_percs,
        "net_profit": humanize.intcomma(net_profit),
        "start": str(start),
        "end": str(end),
        "return_percentage": return_percentage,
        "win_rate_percentage": win_rate_percentage,
        "profit_factor": profit_factor,
        "max_drawdown_percentage": max_drawdown_percentage,
        "average_drawdown_percentage": average_drawdown_percentage,
        "trades": trades,
        "false_buys": false_buys,
        "false_sells": false_sells,
        "percentage_false_trades": percentage_false_trades,
        "equity": equity,
        "trades": stats._trades,
        "bt": bt
    }

In [ ]:
def tot_prof(trades, initial):
    tot_prof = initial
    for perc in trades:
        tot_prof  *= (1 + perc)
        if tot_prof < 0:
            raise Exception('Negative balance')
    return tot_prof

def test_periods(start_end_indices, all_stats = [], actual_cash=1000, per_trade_risk=0.02, leverage=20, hedging=False, crypto_cash_adjustment=100000):
    profit_takeout = []
    bulk_stats = []
    errors = []
    initial_adjusted_equity = crypto_cash_adjustment
    current_adjusted_equity = initial_adjusted_equity
    for start, end in start_end_indices:
        try:
            df_sample = df[start:end].copy()    
            result = backtest_period(RangeBarStrategyPreCalc, df_sample, all_stats, start, end, per_trade_risk=per_trade_risk, leverage=leverage, hedging=hedging, crypto_cash_adjustment=current_adjusted_equity)
            if result['equity'] < initial_adjusted_equity:
                current_adjusted_equity = result['equity']
            else:
                current_adjusted_equity = initial_adjusted_equity
                actual_equity = tot_prof(result['trade_return_percentages'], actual_cash)
                takeout =  actual_equity - actual_cash
                profit_takeout.append([f'{start}-{end}', takeout])
            bulk_stats.append(result)
        except Exception as e:
            errors.append(f'strat: RangeBarStrategyPreCalc,  start: {str(start)}, end: {str(end)}. error: {str(e)}')

    print(f'errors: {len(errors)}')
    print(f'errors: {errors}')
    df_stats = pd.DataFrame(bulk_stats)
    return df_stats, all_stats, profit_takeout, bulk_stats


In [ ]:
per_trade_risk=0.02
actual_cash=1000
leverage=20

df_stats, all_stats, profit_takeout, bulk_stats = test_periods(start_end_indices, actual_cash=actual_cash,  per_trade_risk=per_trade_risk, leverage=20, hedging=False)
print(f'profit_takeout per period:\n')
for pt in profit_takeout:
    period, profit = pt
    print(f'{period}: {humanize.intcomma(round(profit,2))}')
takeout = [x[1] for x in profit_takeout]
print(f'total profit takeout: {humanize.intcomma(sum(takeout))}')


## with filler

2022-04-13 17:32:00-2022-04-30 23:59:00: 406.6  
2022-05-01 00:00:00-2022-05-31 23:54:00: 184,184.06  
2022-06-01 00:00:00-2022-06-30 23:59:01: 280,256.86  
2022-07-01 00:00:00-2022-07-31 23:59:00: 285,264.94  
2022-08-01 00:00:00-2022-08-31 23:46:00: 11,906.28  
2022-09-01 00:00:00-2022-09-30 23:59:00: 16,999.74  
2022-10-01 00:00:00-2022-10-31 23:48:00: 2,965.89  
2022-11-01 00:05:00-2022-11-30 23:54:00: 16,825.25  
2022-12-01 00:05:00-2022-12-31 23:38:01: 2,073.43  
2023-01-01 00:08:00-2023-01-31 23:59:00: 7,633.88  
2023-02-01 00:00:00-2023-02-28 23:34:00: 3,230.02  
2023-03-01 00:16:00-2023-03-31 23:59:00: 10,978.79  
2023-04-01 00:00:00-2023-04-14 16:39:00: 677.23  
total profit takeout: 823,402.9596548487  

In [ ]:
for stat in bulk_stats:
    start = stat['start']
    end = stat['end']
    # stat['bt'].plot(filename=f'../tmp-data/period-plots/{start}-{end}')
    stat['trades'].to_csv(f'../../tmp-data/period-trades/{actual_cash}-{per_trade_risk}-{start}-{end}.csv')

In [ ]:
print(f'all_stats: {len(all_stats)}')
all_stats_str = ""
for stats in all_stats:
    all_stats_str += stats.to_string() + f'\n{stats._trades.to_string()}\n' + "\n\n-------------------------------------------------------------------------------------------------\n\n"

suffix = input_path.split('/')[-1].split('.')[-1]
with open(f'../../tmp-data/all_stats-{suffix}-{actual_cash}-{per_trade_risk}-no_zero_close.txt', 'w') as f:
    f.write(all_stats_str)    

In [ ]:
pf_measure = df_stats['profit_factor'].sum()
pf_measure

In [ ]:
dd_p_measure = df_stats['max_drawdown_percentage'].sum()
dd_p_measure